In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data=pd.read_csv('/kaggle/input/facial-keypoints-detection/training.zip')
test_data=pd.read_csv('/kaggle/input/facial-keypoints-detection/test.zip')

In [ ]:
train_data.head()

Check for missing values

In [ ]:
train_data.isnull().sum()


In [ ]:
null_c=train_data.columns.tolist()
null_c

In [ ]:
for i in null_c:
    train_data[i]=train_data[i].transform(lambda x:x.fillna(x.mean()))

In [ ]:
train_data.isnull().sum()

Extracting images

In [ ]:
train_data['Image']=train_data['Image'].apply(lambda x: np.fromstring(x,sep=' '))

In [ ]:
train_data['Image']

In [ ]:
X_train=np.vstack(train_data['Image'].values)
X_train=X_train.reshape(-1,96,96,1)
X_train=X_train.astype(np.float32)

In [ ]:
X_train.shape

Extracting points

In [ ]:
train_data=train_data.drop(['Image'],axis=1)
Y_train=train_data.values.astype(np.float32)
Y_train.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(X_train[5].reshape(96,96),cmap='gray')
plt.show()

In [ ]:
def show_image(X, Y):
    img = np.copy(X)
    for i in range(0,Y.shape[0],2):
        if 0 < Y[i+1] < 96 and 0 < Y[i] < 96:
            img[int(Y[i+1]),int(Y[i]),0] = 255
    plt.imshow(img[:,:,0])

In [ ]:
# Preview dataset samples
show_image(X_train[5], Y_train[5])

Define our model

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
model=Sequential()
model.add(Convolution2D(32,3,3,input_shape=(96,96,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64,2,2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(128,2,2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
          
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['mae'])

In [ ]:
hist=model.fit(X_train,Y_train,epochs = 100,validation_split = 0.2)

In [ ]:
#hist=model.fit(X_train,Y_train,epochs = 50,batch_size = 256,validation_split = 0.2)

In [ ]:
f=plt.figure()
plt.plot(hist.history['loss'],linewidth=3,label='train')
plt.plot(hist.history['val_loss'],linewidth=3,label='valid')
plt.grid()
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.yscale('log')
plt.show()

load test data

In [ ]:
test_data.shape

In [ ]:
test_data['Image']=test_data['Image'].apply(lambda x: np.fromstring(x,sep=' '))
X_test=np.vstack(test_data['Image'].values)
X_test=X_test.reshape(-1,96,96,1)
X_test=X_test.astype(np.float32)


In [ ]:
plt.imshow(X_test[5].reshape(96,96),cmap = 'gray')
plt.show()

In [ ]:
def show_results(image_index):
    Ypred = model.predict(X_test[image_index:(image_index+1)])
    show_image(X_test[image_index], Ypred[0])

In [ ]:
show_results(5)

In [ ]:
lookid=pd.read_csv('/kaggle/input/facial-keypoints-detection/IdLookupTable.csv')
sample=pd.read_csv('/kaggle/input/facial-keypoints-detection/SampleSubmission.csv')
pred=model.predict(X_test)

In [ ]:
sample.head()

In [ ]:
lookid_list = list(lookid['FeatureName'])
imageID = list(lookid['ImageId']-1)
pre_list = list(pred)

In [ ]:
rowid = lookid['RowId']
rowid=list(rowid)
feature = []
for f in list(lookid['FeatureName']):
    feature.append(lookid_list.index(f))

In [ ]:
preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])
rowid = pd.Series(rowid,name = 'RowId')
loc = pd.Series(preded,name = 'Location')
sub = pd.concat([rowid,loc],axis = 1)

In [ ]:
sub.to_csv('pg_submission.csv',index = False)

References:
[https://www.kaggle.com/karanjakhar/facial-keypoint-detection](http://)